# Setup and Installations

In [ ]:
# If you are using GPU machine, run this cell
import os
import sys
# Replace with your SUMO installation path
# Set the SUMO_HOME environment variable
os.environ['SUMO_HOME'] = '/usr/share/sumo'  # Update this path

sys.path.append(os.path.join(os.environ['SUMO_HOME'], 'tools'))

os.environ['RLLIB_NUM_GPUS'] = '1'  # Set this to the number of GPUs you want to use

In [ ]:
import utils
import env_manager as env_manager
import algo_trainer as algo_trainer
from typing import SupportsIndex

# Training Function

In [ ]:
def chain_training(manager: env_manager, generator: env_manager.EnvManager.env_generator, algo_agent, running_result_grid: list):
    if len(running_result_grid) != 0: 
        # take the best config from the previous training 
        best = running_result_grid[-1].get_best_result(metric="env_runners/episode_reward_mean", mode="max")
        
        # Initialize the environment manager with new route file
        rou, csv = next(generator)
        manager.initialize_env(rou, csv)
        
        # continue the training with the best config
        algo_agent.config = algo_agent.from_dict(best.config)
        algo_agent.build_config(flag=True)
        
    result = algo_agent.train()
    
    return result

def training(num_intersection: int, experiment_type: str, algo_config: str, env_config: str, num_training: SupportsIndex):
    final_running_result = []
    sumo_type = "SingleAgent"
    algo_type = experiment_type.split("_")
     
    if experiment_type.__contains__("Multi"):
        sumo_type = "MultiAgent"
    
    # Initialize the environment manager
    manager = env_manager.EnvManager(f"{sumo_type}Environment", env_config, intersection_id=f"intersection_{num_intersection}")
    generator = manager.env_generator(f"Nets/intersection_{num_intersection}/route_xml_path_intersection_{num_intersection}.txt", algo_name=algo_type[0])
    
    # Initialize the environment manager with new route file
    rou, csv = next(generator)
    manager.initialize_env(rou, csv)
    
    algo_agent = algo_trainer.ALGOTrainer(config_path=algo_config, env_manager=manager, experiment_type=experiment_type)
    algo_agent.build_config()
    for i in range(num_training):
        chain_result = chain_training(manager=manager, generator=generator, algo_agent=algo_agent, running_result_grid=final_running_result)
        if chain_result is not None:
            final_running_result.append(chain_result)
    
    return final_running_result

# Training Configuration

In [ ]:
# Choose which intersection you want to train
num_intersection_to_train = 7

# Choose the experiment_type:
# PPO_SingleAgent | APPO_SingleAgent | DQN_SingleAgent | DDQN_SingleAgent | PPO_MultiAgent | APPO_MultiAgent | DQN_MultiAgent | DDQN_MultiAgent | 
experiment_type = "APPO_MultiAgent"  

# Choose how many training cycles you want to run
num_training_cycles = 1

env_config_file_path = "Config/env_config.json"

ppo_config_file_path = "Config/ppo_config.json"

dqn_config_file_path = "Config/dqn_config.json"

appo_config_file_path = "Config/appo_config.json"

# Training Execution

In [ ]:
results = training(num_intersection=num_intersection_to_train, experiment_type=experiment_type, algo_config=appo_config_file_path, env_config=env_config_file_path, num_training=num_training_cycles)

print(f"Finished training for intersection: {num_intersection_to_train} with {num_training_cycles} training rounds")

In [ ]:
# Notify when done
experiment_date = env_manager.datetime.now().strftime("%m.%d-%H:%M:%S")
message = f'Training for intersection {num_intersection_to_train} with {experiment_type} and {num_training_cycles} from {experiment_date} is done!'  # Replace with your message
recipient = 'eviatar109@icloud.com'  # Replace with your iCloud email
utils.send_imessage(message, recipient)

# Evaluation

In [ ]:
result = results[-1]
from ray.rllib.algorithms.algorithm import Algorithm

best_result = result.get_best_result("env_runners/episode_reward_max", "max")
checkpoint_path = best_result.checkpoint.path
print(f'Best checkpoint path: {checkpoint_path}')

# Load the Algorithm from the checkpoint
algo = Algorithm.from_checkpoint(checkpoint_path)

# Retrieve the current configuration
new_config = algo.config.copy()
new_config["evaluation_duration"] = 2 # Define as many evaluation episodes as you want


# Re-create the algorithm instance with the updated configuration
algo = Algorithm.from_checkpoint(checkpoint_path)
algo.config = new_config

# Evaluate the Algorithm
eval_results = algo.evaluate()
print(eval_results)